<a href="https://colab.research.google.com/github/Jagaya/FCC-Data-Analysis/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

Ressource:
https://www.datacamp.com/community/tutorials/k-nearest-neighbor-classification-scikit-learn

https://datascienceplus.com/building-a-book-recommender-system-the-basics-knn-and-matrix-factorization/

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-11-11 14:53:18--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  72.0MB/s    in 0.3s    

2021-11-11 14:53:19 (72.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# explore data
# df_books.isnull().sum() # one unknown author
# df_ratings.isnull().sum() # no unknown values
df_books[df_books.author.isnull()] # author unknown

,isbn,title,author
187700,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN


In [ ]:
# testing the NearestNeighbor-algorithm

# samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
# neigh = NearestNeighbors(n_neighbors = 2, radius = 0.4)
# neigh.fit(samples)
# print( neigh.kneighbors([[0, 0, 1.3]], 3, return_distance=False) )
# print( neigh.kneighbors([[0, 0, 1.3]], 3, return_distance=True) )

In [ ]:
ratings_isbn = set(df_ratings.isbn.values)
books_isbn = set(df_books.isbn.values)

In [ ]:
print(len(ratings_isbn), len(books_isbn))

340556 271379


In [ ]:
rating_without_books = set()
for i in ratings_isbn:
  if not i in books_isbn:
    rating_without_books.add(i)

In [ ]:
len(rating_without_books)

70386

Remove books with less than 200 ratings and users with less than 100 ratings

- create new column in df_ratings with the number of ratings on each book
- create new column in df_ratings with the number of ratings per user
- create df_filtered only with row where counts fit the condition
- drop the counter-columns
- get rid of duplicate entries via .drop_duplicates(subset=["title","user"], keep="first")
- piv = df.pivot(index='title', columns='user', values='rating')

In [ ]:
df_ratings_count = df_ratings.copy()

In [ ]:
# Books with at least 200 ratings
# isbn have different formats and would need cleaning -> assume those won't impact the filtered results
# counted_ratings = df_ratings.groupby(df_ratings.isbn).count()
# counted_ratings.sort_values("isbn", ascending=False)["isbn"].head()
counted_ratings = df_ratings['isbn'].value_counts()

# counted_users = df_ratings.groupby(df_ratings.user).count()
# counted_users.sort_values("rating", ascending=False)["rating"].head()
counted_users = df_ratings['user'].value_counts()

In [ ]:
counted_ratings[ counted_ratings >= 100].shape

(731,)

In [ ]:
df_filtered_counts = df_ratings.copy()
df_filtered_counts.shape

(1149780, 3)

In [ ]:
# .map() is highly inefficient
# df_ratings_count["rcount"] = df_ratings_count.isbn.map(lambda x: counted_ratings[x])
# df_ratings_count["ucount"] = df_ratings_count.user.map(lambda x: counted_users[x])


df_filtered_counts = df_filtered_counts[ df_filtered_counts["user"].isin(counted_users[ counted_users >= 200 ].index) ]
df_filtered_counts = df_filtered_counts[ df_filtered_counts["isbn"].isin(counted_ratings[ counted_ratings >= 100 ].index)]
df_filtered_counts.shape

(49781, 3)

In [ ]:
df = pd.merge(left=df_filtered_counts, right=df_books, on="isbn")
df.shape

(49517, 5)

In [ ]:
df.drop_duplicates(subset=["title","user"], keep="first", inplace=True)
df.shape

(49136, 5)

In [ ]:
# df = df.dropna(axis = 0, subset = ['title'])

book_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'rating_count'})
     [['title', 'rating_count']]
    )
book_ratingCount.head()

,title,rating_count
0,1984,52
1,1st to Die: A Novel,153
2,2nd Chance,114
3,4 Blondes,72
4,A Beautiful Mind: The Life of Mathematical Gen...,46


In [ ]:
# us_canada_user_rating = us_canada_user_rating.drop_duplicates(['userID', 'bookTitle'])
df_pivot = df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df_csr_matrix = csr_matrix(df_pivot.values)

from sklearn.neighbors import NearestNeighbors

# model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn = NearestNeighbors(metric="cosine",
                             #algorithm="auto",
                             n_neighbors=df_pivot.shape[0])
model_knn.fit(df_csr_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=673, p=2,
                 radius=1.0)

In [ ]:
df_pivot.shape

(673, 888)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = "", model=model_knn):
  distances, indices = model.kneighbors( np.array( df_pivot.loc[book] ).reshape(1,-1) , n_neighbors=6)
  recommends = [
                book,
                [ [df_pivot.index[indices.flatten()[i]], round( distances.flatten()[i], 2) ] for i in range(5,0,-1)]
  ]

  return recommends

get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8],
  ['The Weight of Water', 0.77],
  ['The Surgeon', 0.77],
  ['I Know This Much Is True', 0.77],
  ['The Lovely Bones: A Novel', 0.72]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
